# Necessary libraries

In [57]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import lagrange
from scipy.interpolate import CubicSpline, InterpolatedUnivariateSpline, Akima1DInterpolator
from datetime import datetime, timedelta
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import statsmodels.api as sm
#from datetime import timedelta
#import datetime
#import fbprophet
#from matplotlib import pyplot
from pandas import to_datetime
from statsmodels.tsa.arima_model import ARIMA
#from fbprophet import Prophet
#from pmdarima.arima import auto_arima
#import pmdarima
from scipy import interpolate
#from fbprophet import Prophet
from matplotlib import pyplot
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras import Sequential, layers, callbacks
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
import plotly.graph_objs as go
import sklearn.model_selection
from scipy import stats
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow import keras
from imblearn.over_sampling import SMOTE
from scipy.interpolate import CubicSpline as CS
from sklearn.ensemble import RandomForestClassifier
from scipy import stats
from sklearn.model_selection import GridSearchCV,\
RandomizedSearchCV

In [44]:
#!pip install pystan==2.19.1.1 convertdate==2.4.0 lunarcalendar==0.0.9 holidays==0.13 fbprophet
#!pip install plotly

In [45]:
Wound_assessment = pd.read_csv("2_data_for_imputation_may11.csv")
Wound_assessment=Wound_assessment.drop(["Unnamed: 0"],axis=1)
Wound_assessment

,PatientID,DateAdded_New,WoundNumber,Length,Width,PatientGender,RaceCodeList,patientEthnicity,Age,BMI,SmokingStatus
0,5978401.0,2016-10-26,1,2.00,2.00,Unknown,Unknown,Unknown,71.81,27.5,4.0
1,5978401.0,2016-10-26,1,2.00,2.00,Unknown,Unknown,Unknown,71.81,27.5,4.0
2,5978401.0,2016-10-26,1,2.00,2.00,Unknown,Unknown,Unknown,71.81,27.5,4.0
3,5978401.0,2016-10-26,1,2.00,2.00,Unknown,Unknown,Unknown,71.81,27.5,4.0
4,5978401.0,2016-10-26,1,2.00,2.00,Unknown,Unknown,Unknown,71.81,27.5,4.0
...,...,...,...,...,...,...,...,...,...,...,...
1726829,10219057.0,2023-01-09,2,4.05,6.11,M,White,Non-Hispanic,73.30,44.6,3.0
1726830,10219057.0,2023-01-09,2,4.16,5.59,M,White,Non-Hispanic,73.30,44.6,3.0
1726831,10225856.0,2023-01-10,1,1.36,0.72,M,White,Non-Hispanic,84.90,19.7,3.0
1726832,10223584.0,2023-01-11,1,4.18,3.56,F,Unknown,Non-Hispanic,42.53,34.5,1.0


In [46]:
Wound_assessment["PatientID"]=Wound_assessment["PatientID"].astype("int")
#Wound_assessment["SmokingStatus"]=Wound_assessment["SmokingStatus"].astype("int")
Wound_assessment

,PatientID,DateAdded_New,WoundNumber,Length,Width,PatientGender,RaceCodeList,patientEthnicity,Age,BMI,SmokingStatus
0,5978401,2016-10-26,1,2.00,2.00,Unknown,Unknown,Unknown,71.81,27.5,4.0
1,5978401,2016-10-26,1,2.00,2.00,Unknown,Unknown,Unknown,71.81,27.5,4.0
2,5978401,2016-10-26,1,2.00,2.00,Unknown,Unknown,Unknown,71.81,27.5,4.0
3,5978401,2016-10-26,1,2.00,2.00,Unknown,Unknown,Unknown,71.81,27.5,4.0
4,5978401,2016-10-26,1,2.00,2.00,Unknown,Unknown,Unknown,71.81,27.5,4.0
...,...,...,...,...,...,...,...,...,...,...,...
1726829,10219057,2023-01-09,2,4.05,6.11,M,White,Non-Hispanic,73.30,44.6,3.0
1726830,10219057,2023-01-09,2,4.16,5.59,M,White,Non-Hispanic,73.30,44.6,3.0
1726831,10225856,2023-01-10,1,1.36,0.72,M,White,Non-Hispanic,84.90,19.7,3.0
1726832,10223584,2023-01-11,1,4.18,3.56,F,Unknown,Non-Hispanic,42.53,34.5,1.0


In [47]:
df3=Wound_assessment
df3["Area"]=df3["Length"]*df3["Width"]
df3=df3.reset_index(drop=True)
Wound_assessment=df3
Wound_assessment

,PatientID,DateAdded_New,WoundNumber,Length,Width,PatientGender,RaceCodeList,patientEthnicity,Age,BMI,SmokingStatus,Area
0,5978401,2016-10-26,1,2.00,2.00,Unknown,Unknown,Unknown,71.81,27.5,4.0,4.0000
1,5978401,2016-10-26,1,2.00,2.00,Unknown,Unknown,Unknown,71.81,27.5,4.0,4.0000
2,5978401,2016-10-26,1,2.00,2.00,Unknown,Unknown,Unknown,71.81,27.5,4.0,4.0000
3,5978401,2016-10-26,1,2.00,2.00,Unknown,Unknown,Unknown,71.81,27.5,4.0,4.0000
4,5978401,2016-10-26,1,2.00,2.00,Unknown,Unknown,Unknown,71.81,27.5,4.0,4.0000
...,...,...,...,...,...,...,...,...,...,...,...,...
1726829,10219057,2023-01-09,2,4.05,6.11,M,White,Non-Hispanic,73.30,44.6,3.0,24.7455
1726830,10219057,2023-01-09,2,4.16,5.59,M,White,Non-Hispanic,73.30,44.6,3.0,23.2544
1726831,10225856,2023-01-10,1,1.36,0.72,M,White,Non-Hispanic,84.90,19.7,3.0,0.9792
1726832,10223584,2023-01-11,1,4.18,3.56,F,Unknown,Non-Hispanic,42.53,34.5,1.0,14.8808


In [48]:
Wound_assessment["SmokingStatus"]=Wound_assessment["SmokingStatus"].replace({np.nan:0.0})
Wound_assessment["BMI"]=Wound_assessment["BMI"].replace({np.nan:Wound_assessment["BMI"].mean()})
Wound_assessment

,PatientID,DateAdded_New,WoundNumber,Length,Width,PatientGender,RaceCodeList,patientEthnicity,Age,BMI,SmokingStatus,Area
0,5978401,2016-10-26,1,2.00,2.00,Unknown,Unknown,Unknown,71.81,27.500000,4.0,4.0000
1,5978401,2016-10-26,1,2.00,2.00,Unknown,Unknown,Unknown,71.81,27.500000,4.0,4.0000
2,5978401,2016-10-26,1,2.00,2.00,Unknown,Unknown,Unknown,71.81,27.500000,4.0,4.0000
3,5978401,2016-10-26,1,2.00,2.00,Unknown,Unknown,Unknown,71.81,27.500000,4.0,4.0000
4,5978401,2016-10-26,1,2.00,2.00,Unknown,Unknown,Unknown,71.81,27.500000,4.0,4.0000
...,...,...,...,...,...,...,...,...,...,...,...,...
1726829,10219057,2023-01-09,2,4.05,6.11,M,White,Non-Hispanic,73.30,44.600000,3.0,24.7455
1726830,10219057,2023-01-09,2,4.16,5.59,M,White,Non-Hispanic,73.30,44.600000,3.0,23.2544
1726831,10225856,2023-01-10,1,1.36,0.72,M,White,Non-Hispanic,84.90,19.700000,3.0,0.9792
1726832,10223584,2023-01-11,1,4.18,3.56,F,Unknown,Non-Hispanic,42.53,34.500000,1.0,14.8808


In [ ]:
list_patients=Wound_assessment["PatientID"].unique()
print("There are",len(list_patients),"number of patients whose wound assessment information is found in the merged dataframe.")
list_count_patient_for_forecasting=[]
unique_wound_number=0
for i in range(len(list_patients)):
    #print(i)
    patient=list_patients[i]
    df_patient=Wound_assessment.query("PatientID == @patient")
    if df_patient.shape[0]>1:
        #print(i)
        wound_list_patient=df_patient["WoundNumber"].unique().tolist()
        print("For patient",i,"number of unique wounds",len(wound_list_patient))
        unique_wound_number=unique_wound_number+len(wound_list_patient)
        list_count_patient_for_forecasting.append(i)
        #display(df_patient)
print("Forecasting can be done for",len(list_count_patient_for_forecasting),"number of patients, because they have more than one wound assessments. Others do not.")
print("Number of unique wounds for",len(list_count_patient_for_forecasting),"patients is",unique_wound_number)

In [50]:
## Making two separate dataframes for numeric and categorical variables and imputing the values

In [51]:
Wound_assessment_numeric=Wound_assessment[["PatientID", "DateAdded_New","WoundNumber","Length","Width","Age","BMI"]]
Wound_assessment_numeric= Wound_assessment_numeric.groupby('PatientID').agg('mean')
Wound_assessment_numeric=Wound_assessment_numeric.reset_index()
Wound_assessment_numeric

,PatientID,WoundNumber,Length,Width,Age,BMI
0,5117750,1.000000,5.000000,3.000000,33.940000,19.200000
1,5474795,1.818182,2.818182,2.727273,71.047273,24.800000
2,5476216,4.419753,5.872469,5.403210,69.968272,20.800000
3,5640543,1.750000,2.300000,4.550000,50.670000,34.264272
4,5652673,1.000000,0.868750,2.506250,19.521250,22.900000
...,...,...,...,...,...,...
7394,10229542,1.500000,1.750000,1.200000,51.770000,24.100000
7395,10230613,1.000000,3.300000,6.100000,18.800000,25.500000
7396,10231328,1.000000,1.000000,1.500000,48.920000,38.400000
7397,10231478,1.500000,2.400000,0.450000,66.500000,29.200000


In [52]:
Wound_assessment_categorical=Wound_assessment[["PatientID","PatientGender","RaceCodeList","patientEthnicity","SmokingStatus"]]
Wound_assessment_categorical= Wound_assessment_categorical.groupby('PatientID').agg('max')
Wound_assessment_categorical=Wound_assessment_categorical.reset_index()
Wound_assessment_categorical

,PatientID,PatientGender,RaceCodeList,patientEthnicity,SmokingStatus
0,5117750,F,Black,Non-Hispanic,4.0
1,5474795,Unknown,Unknown,Unknown,3.0
2,5476216,Unknown,Unknown,Unknown,4.0
3,5640543,M,Unknown,Unknown,0.0
4,5652673,F,White,Non-Hispanic,4.0
...,...,...,...,...,...
7394,10229542,M,Unknown,Non-Hispanic,0.0
7395,10230613,F,Unknown,Unknown,0.0
7396,10231328,M,Unknown,Unknown,1.0
7397,10231478,M,Unknown,Unknown,4.0


### Merging and Replacing all Unknowns with np.nan

In [54]:
#pd.merge(data_numeric, data_coded,left_index=True, right_index=True)
df2 = pd.merge(Wound_assessment_numeric, Wound_assessment_categorical, on='PatientID')
df2=df2.reset_index()
df2=df2.drop(["index"],axis=1)
df2

print("Value counts for Gender:")
print(df2["PatientGender"].value_counts())
print("\n")
print("Value counts for Race:")
print(df2["RaceCodeList"].value_counts())
print("\n")
print("Value counts for Ethnicity:")
print(df2["patientEthnicity"].value_counts())

df2=df2.replace({"Unknown":np.nan})
df2["PatientGender"]=df2["PatientGender"].replace({"M":0,"F":1})
df2["RaceCodeList"]=df2["RaceCodeList"].replace({"White":0,"Black":1,"Asian":2,"ASKU":3,
                                                 "American Indian or Alaska Native":4,
                                                 "Native Hawaiian or Other Pacific Islander":5,
                                                 "Other":6})
df2["patientEthnicity"]=df2["patientEthnicity"].replace({"Non-Hispanic":0,"Hispanic":1,"Patient Declined":2,"ASKU":3})

print("Number of PatientGender need be imputed:",df2["PatientGender"].shape[0]-df2["PatientGender"].dropna().shape[0])
print("Number of RaceCodeList need be imputed:",df2["RaceCodeList"].shape[0]-df2["RaceCodeList"].dropna().shape[0])
print("Number of patientEthnicity need be imputed:",df2["patientEthnicity"].shape[0]-df2["patientEthnicity"].dropna().shape[0])

Value counts for Gender:
M          3833
F          3556
Unknown      10
Name: PatientGender, dtype: int64


Value counts for Race:
Unknown                                      3514
White                                        3240
Black                                         588
Asian                                          24
ASKU                                           22
American Indian or Alaska Native                6
Native Hawaiian or Other Pacific Islander       3
Other                                           2
Name: RaceCodeList, dtype: int64


Value counts for Ethnicity:
Non-Hispanic        4930
Unknown             2368
Hispanic              92
Patient Declined       8
ASKU                   1
Name: patientEthnicity, dtype: int64
Number of PatientGender need be imputed: 10
Number of RaceCodeList need be imputed: 3514
Number of patientEthnicity need be imputed: 2368


## Imputing PatientGender

In [41]:
#default without hyperparameter tuning

#pd.merge(data_numeric, data_coded,left_index=True, right_index=True)
df2 = pd.merge(Wound_assessment_numeric, Wound_assessment_categorical, on='PatientID')
df2=df2.reset_index()
df2=df2.drop(["index"],axis=1)
df2

print("Value counts for Gender:")
print(df2["PatientGender"].value_counts())
print("\n")
print("Value counts for Race:")
print(df2["RaceCodeList"].value_counts())
print("\n")
print("Value counts for Ethnicity:")
print(df2["patientEthnicity"].value_counts())

df2=df2.replace({"Unknown":np.nan})
df2["PatientGender"]=df2["PatientGender"].replace({"M":0,"F":1})
df2["RaceCodeList"]=df2["RaceCodeList"].replace({"White":0,"Black":1,"Asian":2,"ASKU":3,
                                                 "American Indian or Alaska Native":4,
                                                 "Native Hawaiian or Other Pacific Islander":5,
                                                 "Other":6})
df2["patientEthnicity"]=df2["patientEthnicity"].replace({"Non-Hispanic":0,"Hispanic":1,"Patient Declined":2,"ASKU":3})

print("Number of PatientGender need be imputed:",df2["PatientGender"].shape[0]-df2["PatientGender"].dropna().shape[0])
print("Number of RaceCodeList need be imputed:",df2["RaceCodeList"].shape[0]-df2["RaceCodeList"].dropna().shape[0])
print("Number of patientEthnicity need be imputed:",df2["patientEthnicity"].shape[0]-df2["patientEthnicity"].dropna().shape[0])
#data=df2
df3=df2
#df2=df2.drop(["index"],axis=1)
df3=df3.sort_values(by=['PatientGender'])

df3["Length"]=df3["Length"].fillna(value=df3["Length"].mean())
df3["Width"]=df3["Width"].fillna(value=df3["Width"].mean())
#df3["Depth"]=df3["Depth"].fillna(value=df3["Depth"].mean())
df3["RaceCodeList"]=df3["RaceCodeList"].fillna(value=df3["RaceCodeList"].mean())
df3["patientEthnicity"]=df3["patientEthnicity"].fillna(value=df3["patientEthnicity"].mean())



test_set=df3.tail(df3["PatientGender"].shape[0]-df3["PatientGender"].dropna().shape[0])
test_set
#new_df = df.drop(df1.index)
train_set=df3.drop(test_set.index)
train_set

X_train=train_set.drop(["PatientID","PatientGender"],axis=1)
X_test=test_set.drop(["PatientID","PatientGender"],axis=1)
y_train=train_set["PatientGender"]

X_train=X_train.values
y_train=y_train.values

# define oversampling strategy
SMOTE = SMOTE()

# fit and apply the transform
X_train_SMOTE, y_train_SMOTE = SMOTE.fit_resample(X_train, y_train)

rf = RandomForestClassifier()
rf.fit(X_train_SMOTE, y_train_SMOTE)

y_pred = rf.predict(X_test)
test_set["PatientGender"]=y_pred

new_df_gender=pd.concat([train_set,test_set],axis=0)
new_df_gender=new_df_gender[["PatientID","PatientGender"]]
new_df_gender.rename({'PatientGender': 'PatientGender_new'}, axis=1, inplace=True)


dff=pd.merge(df2,new_df_gender,on="PatientID")
dff=dff.drop(["PatientGender"],axis=1)
df2=dff
df2

Value counts for Gender:
M          3833
F          3556
Unknown      10
Name: PatientGender, dtype: int64


Value counts for Race:
Unknown                                      3514
White                                        3240
Black                                         588
Asian                                          24
ASKU                                           22
American Indian or Alaska Native                6
Native Hawaiian or Other Pacific Islander       3
Other                                           2
Name: RaceCodeList, dtype: int64


Value counts for Ethnicity:
Non-Hispanic        4930
Unknown             2368
Hispanic              92
Patient Declined       8
ASKU                   1
Name: patientEthnicity, dtype: int64
Number of PatientGender need be imputed: 10
Number of RaceCodeList need be imputed: 3514
Number of patientEthnicity need be imputed: 2368


,PatientID,WoundNumber,Length,Width,Age,BMI,RaceCodeList,patientEthnicity,SmokingStatus,PatientGender_new
0,5117750,1.000000,5.000000,3.000000,33.940000,19.200000,1.0,0.0,4.0,1.0
1,5474795,1.818182,2.818182,2.727273,71.047273,24.800000,NaN,NaN,3.0,0.0
2,5476216,4.419753,5.872469,5.403210,69.968272,20.800000,NaN,NaN,4.0,1.0
3,5640543,1.750000,2.300000,4.550000,50.670000,34.264272,NaN,NaN,0.0,0.0
4,5652673,1.000000,0.868750,2.506250,19.521250,22.900000,0.0,0.0,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...
7394,10229542,1.500000,1.750000,1.200000,51.770000,24.100000,NaN,0.0,0.0,0.0
7395,10230613,1.000000,3.300000,6.100000,18.800000,25.500000,NaN,NaN,0.0,1.0
7396,10231328,1.000000,1.000000,1.500000,48.920000,38.400000,NaN,NaN,1.0,0.0
7397,10231478,1.500000,2.400000,0.450000,66.500000,29.200000,NaN,NaN,4.0,0.0


In [31]:
print(train_set["PatientGender"].value_counts(normalize=True).mul(100).round(10).astype(str) + '%')
print(df2["PatientGender_new"].value_counts(normalize=True).mul(100).round(10).astype(str) + '%')

0.0    51.8744079036%
1.0    48.1255920964%
Name: PatientGender, dtype: object
0.0    51.8989052575%
1.0    48.1010947425%
Name: PatientGender_new, dtype: object


In [32]:
param_grid = {
    'n_estimators': [25, 50, 100, 150],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [3, 6, 9],
    'max_leaf_nodes': [3, 6, 9],
}

In [33]:
grid_search = GridSearchCV(RandomForestClassifier(),
                           param_grid=param_grid)
grid_search.fit(X_train_SMOTE, y_train_SMOTE)
print(grid_search.best_estimator_)

RandomForestClassifier(max_depth=6, max_features=None, max_leaf_nodes=9,
                       n_estimators=25)


In [55]:
#with hyperparameter tuning
#data=df2
df3=df2
#df2=df2.drop(["index"],axis=1)
df3=df3.sort_values(by=['PatientGender'])

df3["Length"]=df3["Length"].fillna(value=df3["Length"].mean())
df3["Width"]=df3["Width"].fillna(value=df3["Width"].mean())
#df3["Depth"]=df3["Depth"].fillna(value=df3["Depth"].mean())
df3["RaceCodeList"]=df3["RaceCodeList"].fillna(value=df3["RaceCodeList"].mean())
df3["patientEthnicity"]=df3["patientEthnicity"].fillna(value=df3["patientEthnicity"].mean())



test_set=df3.tail(df3["PatientGender"].shape[0]-df3["PatientGender"].dropna().shape[0])
test_set
#new_df = df.drop(df1.index)
train_set=df3.drop(test_set.index)
train_set

X_train=train_set.drop(["PatientID","PatientGender"],axis=1)
X_test=test_set.drop(["PatientID","PatientGender"],axis=1)
y_train=train_set["PatientGender"]

X_train=X_train.values
y_train=y_train.values

# define oversampling strategy
SMOTE = SMOTE()

# fit and apply the transform
X_train_SMOTE, y_train_SMOTE = SMOTE.fit_resample(X_train, y_train)

model_grid = RandomForestClassifier(max_depth=6,
                                    max_features="log2",
                                    max_leaf_nodes=9,
                                    n_estimators=25)
model_grid.fit(X_train_SMOTE, y_train_SMOTE)

y_pred = rf.predict(X_test)
test_set["PatientGender"]=y_pred

new_df_gender=pd.concat([train_set,test_set],axis=0)
new_df_gender=new_df_gender[["PatientID","PatientGender"]]
new_df_gender.rename({'PatientGender': 'PatientGender_new'}, axis=1, inplace=True)


dff=pd.merge(df2,new_df_gender,on="PatientID")
dff=dff.drop(["PatientGender"],axis=1)
df2=dff
df2

,PatientID,WoundNumber,Length,Width,Age,BMI,RaceCodeList,patientEthnicity,SmokingStatus,PatientGender_new
0,5117750,1.000000,5.000000,3.000000,33.940000,19.200000,1.0,0.0,4.0,1.0
1,5474795,1.818182,2.818182,2.727273,71.047273,24.800000,NaN,NaN,3.0,0.0
2,5476216,4.419753,5.872469,5.403210,69.968272,20.800000,NaN,NaN,4.0,1.0
3,5640543,1.750000,2.300000,4.550000,50.670000,34.264272,NaN,NaN,0.0,0.0
4,5652673,1.000000,0.868750,2.506250,19.521250,22.900000,0.0,0.0,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...
7394,10229542,1.500000,1.750000,1.200000,51.770000,24.100000,NaN,0.0,0.0,0.0
7395,10230613,1.000000,3.300000,6.100000,18.800000,25.500000,NaN,NaN,0.0,1.0
7396,10231328,1.000000,1.000000,1.500000,48.920000,38.400000,NaN,NaN,1.0,0.0
7397,10231478,1.500000,2.400000,0.450000,66.500000,29.200000,NaN,NaN,4.0,0.0


In [56]:
print(train_set["PatientGender"].value_counts(normalize=True).mul(100).round(10).astype(str) + '%')
print(df2["PatientGender_new"].value_counts(normalize=True).mul(100).round(10).astype(str) + '%')

0.0    51.8744079036%
1.0    48.1255920964%
Name: PatientGender, dtype: object
0.0    51.9124205974%
1.0    48.0875794026%
Name: PatientGender_new, dtype: object


## Imputing RaceCodeList

In [43]:
from imblearn.over_sampling import SMOTE
#data=df2
df3=df2
#df2=df2.drop(["index"],axis=1)
df3=df3.sort_values(by=['RaceCodeList'])

df3["Length"]=df3["Length"].fillna(value=df3["Length"].mean())
df3["Width"]=df3["Width"].fillna(value=df3["Width"].mean())
#df3["Depth"]=df3["Depth"].fillna(value=df3["Depth"].mean())
df3["patientEthnicity"]=df3["patientEthnicity"].fillna(value=df3["patientEthnicity"].mean())



test_set=df3.tail(df3["RaceCodeList"].shape[0]-df3["RaceCodeList"].dropna().shape[0])
test_set
#new_df = df.drop(df1.index)
train_set=df3.drop(test_set.index)
train_set

X_train=train_set.drop(["PatientID","RaceCodeList"],axis=1)
X_test=test_set.drop(["PatientID","RaceCodeList"],axis=1)
y_train=train_set["RaceCodeList"]

X_train=X_train.values
y_train=y_train.values

# define oversampling strategy

SMOTE = SMOTE(k_neighbors=1)
#SMOTE=SMOTE()
# fit and apply the transform
#X_train_SMOTE, y_train_SMOTE = SMOTE.fit_resample(X_train, y_train)

X_train_SMOTE, y_train_SMOTE = SMOTE.fit_resample(X_train, y_train)


rf = RandomForestClassifier()
rf.fit(X_train_SMOTE, y_train_SMOTE)

y_pred = rf.predict(X_test)
test_set["RaceCodeList"]=y_pred

new_df_gender=pd.concat([train_set,test_set],axis=0)
new_df_gender=new_df_gender[["PatientID","RaceCodeList"]]
new_df_gender.rename({'RaceCodeList': 'RaceCodeList_new'}, axis=1, inplace=True)


dff=pd.merge(df2,new_df_gender,on="PatientID")
dff=dff.drop(["RaceCodeList"],axis=1)
df2=dff
df2

,PatientID,WoundNumber,Length,Width,Age,BMI,patientEthnicity,SmokingStatus,PatientGender_new,RaceCodeList_new
0,5117750,1.000000,5.000000,3.000000,33.940000,19.200000,0.0,4.0,1.0,1.0
1,5474795,1.818182,2.818182,2.727273,71.047273,24.800000,NaN,3.0,0.0,0.0
2,5476216,4.419753,5.872469,5.403210,69.968272,20.800000,NaN,4.0,1.0,0.0
3,5640543,1.750000,2.300000,4.550000,50.670000,34.264272,NaN,0.0,0.0,0.0
4,5652673,1.000000,0.868750,2.506250,19.521250,22.900000,0.0,4.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
7394,10229542,1.500000,1.750000,1.200000,51.770000,24.100000,0.0,0.0,0.0,0.0
7395,10230613,1.000000,3.300000,6.100000,18.800000,25.500000,NaN,0.0,1.0,0.0
7396,10231328,1.000000,1.000000,1.500000,48.920000,38.400000,NaN,1.0,0.0,0.0
7397,10231478,1.500000,2.400000,0.450000,66.500000,29.200000,NaN,4.0,0.0,0.0


In [44]:
print(train_set["RaceCodeList"].value_counts(normalize=True).mul(100).round(10).astype(str) + '%')
print(df2["RaceCodeList_new"].value_counts(normalize=True).mul(100).round(10).astype(str) + '%')

0.0    83.3976833977%
1.0    15.1351351351%
2.0     0.6177606178%
3.0     0.5662805663%
4.0     0.1544401544%
5.0     0.0772200772%
6.0     0.0514800515%
Name: RaceCodeList, dtype: object
0.0    89.6067036086%
1.0     9.5688606568%
2.0     0.3513988377%
3.0     0.3243681579%
4.0     0.0810920395%
5.0     0.0405460197%
6.0     0.0270306798%
Name: RaceCodeList_new, dtype: object


## Imputing patientEthnicity

In [45]:
from imblearn.over_sampling import SMOTE
#data=df2
df3=df2
#df2=df2.drop(["index"],axis=1)
df3=df3.sort_values(by=['patientEthnicity'])

df3["Length"]=df3["Length"].fillna(value=df3["Length"].mean())
df3["Width"]=df3["Width"].fillna(value=df3["Width"].mean())
#df3["Depth"]=df3["Depth"].fillna(value=df3["Depth"].mean())



test_set=df3.tail(df3["patientEthnicity"].shape[0]-df3["patientEthnicity"].dropna().shape[0])
test_set
#new_df = df.drop(df1.index)
train_set=df3.drop(test_set.index)
train_set

X_train=train_set.drop(["PatientID","patientEthnicity"],axis=1)
X_test=test_set.drop(["PatientID","patientEthnicity"],axis=1)
y_train=train_set["patientEthnicity"]

X_train=X_train.values
y_train=y_train.values

# # define oversampling strategy
# SMOTE = SMOTE(k_neighbors=2)

# # fit and apply the transform
# X_train_SMOTE, y_train_SMOTE = SMOTE.fit_resample(X_train, y_train)

rf = RandomForestClassifier()
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
test_set["patientEthnicity"]=y_pred

new_df_gender=pd.concat([train_set,test_set],axis=0)
new_df_gender=new_df_gender[["PatientID","patientEthnicity"]]
new_df_gender.rename({'patientEthnicity': 'patientEthnicity_new'}, axis=1, inplace=True)


dff=pd.merge(df2,new_df_gender,on="PatientID")
dff=dff.drop(["patientEthnicity"],axis=1)
df2=dff
df2

,PatientID,WoundNumber,Length,Width,Age,BMI,SmokingStatus,PatientGender_new,RaceCodeList_new,patientEthnicity_new
0,5117750,1.000000,5.000000,3.000000,33.940000,19.200000,4.0,1.0,1.0,0.0
1,5474795,1.818182,2.818182,2.727273,71.047273,24.800000,3.0,0.0,0.0,0.0
2,5476216,4.419753,5.872469,5.403210,69.968272,20.800000,4.0,1.0,0.0,0.0
3,5640543,1.750000,2.300000,4.550000,50.670000,34.264272,0.0,0.0,0.0,0.0
4,5652673,1.000000,0.868750,2.506250,19.521250,22.900000,4.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
7394,10229542,1.500000,1.750000,1.200000,51.770000,24.100000,0.0,0.0,0.0,0.0
7395,10230613,1.000000,3.300000,6.100000,18.800000,25.500000,0.0,1.0,0.0,0.0
7396,10231328,1.000000,1.000000,1.500000,48.920000,38.400000,1.0,0.0,0.0,0.0
7397,10231478,1.500000,2.400000,0.450000,66.500000,29.200000,4.0,0.0,0.0,0.0


In [46]:
print(train_set["patientEthnicity"].value_counts(normalize=True).mul(100).round(10).astype(str) + '%')
print(df2["patientEthnicity_new"].value_counts(normalize=True).mul(100).round(10).astype(str) + '%')

0.0    97.9924468297%
1.0     1.8286622938%
2.0     0.1590141125%
3.0     0.0198767641%
Name: patientEthnicity, dtype: object
0.0    98.634950669%
1.0    1.2434112718%
2.0    0.1081227193%
3.0    0.0135153399%
Name: patientEthnicity_new, dtype: object


In [47]:
print("Value counts for Gender:")
print(df2["PatientGender_new"].value_counts())
print("\n")
print("Value counts for Race:")
print(df2["RaceCodeList_new"].value_counts())
print("\n")
print("Value counts for Ethnicity:")
print(df2["patientEthnicity_new"].value_counts())

Value counts for Gender:
0.0    3841
1.0    3558
Name: PatientGender_new, dtype: int64


Value counts for Race:
0.0    6630
1.0     708
2.0      26
3.0      24
4.0       6
5.0       3
6.0       2
Name: RaceCodeList_new, dtype: int64


Value counts for Ethnicity:
0.0    7298
1.0      92
2.0       8
3.0       1
Name: patientEthnicity_new, dtype: int64


## Now merging with the Wound_assessment table with the imputed information

In [49]:
new_df_2=df2[["PatientID","PatientGender_new","RaceCodeList_new","patientEthnicity_new"]]
new_df=pd.merge(Wound_assessment,new_df_2,on="PatientID")
#new_df
new_df

,PatientID,DateAdded_New,WoundNumber,Length,Width,PatientGender,RaceCodeList,patientEthnicity,Age,BMI,SmokingStatus,Area,PatientGender_new,RaceCodeList_new,patientEthnicity_new
0,5978401,2016-10-26,1,2.00,2.00,Unknown,Unknown,Unknown,71.81,27.500000,4.0,4.0000,0.0,0.0,0.0
1,5978401,2016-10-26,1,2.00,2.00,Unknown,Unknown,Unknown,71.81,27.500000,4.0,4.0000,0.0,0.0,0.0
2,5978401,2016-10-26,1,2.00,2.00,Unknown,Unknown,Unknown,71.81,27.500000,4.0,4.0000,0.0,0.0,0.0
3,5978401,2016-10-26,1,2.00,2.00,Unknown,Unknown,Unknown,71.81,27.500000,4.0,4.0000,0.0,0.0,0.0
4,5978401,2016-10-26,1,2.00,2.00,Unknown,Unknown,Unknown,71.81,27.500000,4.0,4.0000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1726829,10219057,2023-01-09,2,4.05,6.11,M,White,Non-Hispanic,73.30,44.600000,3.0,24.7455,0.0,0.0,0.0
1726830,10219057,2023-01-09,2,4.16,5.59,M,White,Non-Hispanic,73.30,44.600000,3.0,23.2544,0.0,0.0,0.0
1726831,10225856,2023-01-10,1,1.36,0.72,M,White,Non-Hispanic,84.90,19.700000,3.0,0.9792,0.0,0.0,0.0
1726832,10223584,2023-01-11,1,4.18,3.56,F,Unknown,Non-Hispanic,42.53,34.500000,1.0,14.8808,1.0,0.0,0.0


In [50]:
new_df["PatientGender_new"]=new_df["PatientGender_new"].replace({0.0:"M",1.0:"F"})
new_df["RaceCodeList_new"]=new_df["RaceCodeList_new"].replace({0.0:"White",1.0:"Black",2.0:"Asian",3.0:"ASKU",
                                                 4.0:"American Indian or Alaska Native",
                                                 5.0:"Native Hawaiian or Other Pacific Islander",
                                                 6.0:"Other"})
new_df["patientEthnicity_new"]=new_df["patientEthnicity_new"].replace({0.0:"Non-Hispanic",1.0:"Hispanic",2.0:"Patient Declined",3.0:"ASKU"})
new_df=new_df.drop(["PatientGender","RaceCodeList","patientEthnicity"],axis=1)
new_df.isnull().sum()

PatientID               0
DateAdded_New           0
WoundNumber             0
Length                  0
Width                   0
Age                     0
BMI                     0
SmokingStatus           0
Area                    0
PatientGender_new       0
RaceCodeList_new        0
patientEthnicity_new    0
dtype: int64

In [ ]:
Wound_assessment=new_df
list_patients=Wound_assessment["PatientID"].unique()

unique_wound_number=0
for i in range(len(list_patients)):
    #print(i)
#     if i>9:
#         break
    patient=list_patients[i]
    df_patient=Wound_assessment.query("PatientID == @patient")
    if df_patient.shape[0]>1:
        #print(i)
        wounds_of_list_patient=df_patient["WoundNumber"].unique().tolist()
        #print(wounds_of_list_patient)
        indivi_wound=0
        for j in range(len(wounds_of_list_patient)):
            wound_number= wounds_of_list_patient[j]
            df_patient_1_wound=df_patient.query("WoundNumber==@wound_number")
            #display(df_patient_1_wound)
            if df_patient_1_wound.shape[0]>1:
                if df_patient_1_wound.dropna().shape[0]>=1:
                    indivi_wound=indivi_wound+1
        print("For patient",i+1,"number of unique wounds for which forecasting can be done:",indivi_wound)
#                     #

#                     w_length=pd.Series(df_patient_1_wound["Length"].values)
#                     y = w_length.dropna().values
#                     x=w_length.dropna().index.values
#                     cs = CS(x,y)
#                     x1=w_length.index.values
#                     y1=cs(w_length.index.values)
#                     df_patient_1_wound["Length"]=y1


#                     w_width=pd.Series(df_patient_1_wound["Width"].values)
#                     y = w_width.dropna().values
#                     x=w_width.dropna().index.values
#                     cs = CS(x,y)
#                     x1=w_width.index.values
#                     y1=cs(w_width.index.values)
#                     df_patient_1_wound["Width"]=y1
                    
#                     w_depth=pd.Series(df_patient_1_wound["Depth"].values)
#                     y = w_depth.dropna().values
#                     x=w_depth.dropna().index.values
#                     cs = CS(x,y)
#                     x1=w_depth.index.values
#                     y1=cs(w_depth.index.values)
#                     df_patient_1_wound["Depth"]=y1
                    
                    
#                     display(df_patient_1_wound)
        #print("For patient",i,"number of unique wounds",len(wound_list_patient))
        #for j in range(df_patient_specific_wound=
        unique_wound_number=unique_wound_number+indivi_wound
        #list_count_patient_for_forecasting.append(i)
        #display(df_patient)
print("Total unique wounds:",unique_wound_number)

In [52]:
new_df

,PatientID,DateAdded_New,WoundNumber,Length,Width,Age,BMI,SmokingStatus,Area,PatientGender_new,RaceCodeList_new,patientEthnicity_new
0,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
1,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
2,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
3,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
4,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
...,...,...,...,...,...,...,...,...,...,...,...,...
1726829,10219057,2023-01-09,2,4.05,6.11,73.30,44.600000,3.0,24.7455,M,White,Non-Hispanic
1726830,10219057,2023-01-09,2,4.16,5.59,73.30,44.600000,3.0,23.2544,M,White,Non-Hispanic
1726831,10225856,2023-01-10,1,1.36,0.72,84.90,19.700000,3.0,0.9792,M,White,Non-Hispanic
1726832,10223584,2023-01-11,1,4.18,3.56,42.53,34.500000,1.0,14.8808,F,White,Non-Hispanic


In [53]:
new_df.to_csv("3_imputed_data_for_cleaning.csv")

## Checking if there is statistically significant difference in the race distrbution before and after imputation

In [2]:
race_dis=pd.read_csv("race_dis.csv")
race_dis

,Unnamed: 0,Count before impute,Count after impute
0,White,3240,6630
1,Black,588,708
2,Asian,24,26
3,ASKU,22,24
4,American Indian or Alaska Native,6,6
5,Native Hawaiian or Other Pacific Islander,3,3
6,Other,2,2


In [4]:
x=race_dis["Count before impute"].values
x

array([3240,  588,   24,   22,    6,    3,    2])

In [5]:
y=race_dis["Count after impute"].values
y

array([6630,  708,   26,   24,    6,    3,    2])

In [6]:
stats.kruskal(x, y)

KruskalResult(statistic=0.10294583465314983, pvalue=0.748322752899448)